## 1. prepare the weather data

In [63]:
import csv, re

### test regex: match and groups

In [64]:
regex = r'(\d{4})(\d{2})(\d{2})'
m = re.match(regex, '20111101')

print("m.groups():", m.groups())
print("m.group(1):", m.group(1))
print("m.group(2):", m.group(2))
print("m.group(3):", m.group(3))

m.groups(): ('2011', '11', '01')
m.group(1): 2011
m.group(2): 11
m.group(3): 01


### test DictReader, and try to modify string with re.match.group(index)

In [48]:
with open('data/weather.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row)
        print("date:", row['日期'])
        print("length of row：", len(row))
        m = re.match(regex, row['日期'])
        print("m.groups():", m.groups())
        
        row['日期'] = '-'.join(list(m.groups()))
        print("new date:", row['日期'])
        print(row)
        break

{'日照时数 /0.1小时%': '92', '送端': '133427.912', '平均相对湿度/ 1% ': '71', '日期': '20111101', '受端': '127984.718', '20-20时降水量/ 0.1mm': '0', '平均气温/0.1℃': '135'}
date: 20111101
length of row： 7
m.groups(): ('2011', '11', '01')
new date: 2011-11-01
{'日照时数 /0.1小时%': '92', '送端': '133427.912', '平均相对湿度/ 1% ': '71', '日期': '2011-11-01', '受端': '127984.718', '20-20时降水量/ 0.1mm': '0', '平均气温/0.1℃': '135'}


### do the real work

In [65]:
rows = []
headers = []

with open('data/weather.csv') as csvfile:
    spamreader = csv.DictReader(csvfile)
    headers = spamreader.fieldnames
    for row in spamreader:
        if row['日照时数 /0.1小时%']:
            m = re.match(regex, row['日期'])
            row['日期'] = '-'.join(list(m.groups()))
            rows.append(row)

### write to csv file

In [66]:
with open('data/weather.formated.csv','w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

## 2. join weather data and defect data

In [3]:
import pandas as pd

In [4]:
df1 = pd.read_csv('data/weather.formated.csv')
df2 = pd.read_csv('data/defs.csv')
df3 = pd.merge(df1, df2, left_on='日期', right_on='T_DATE')
del(df3['T_DATE'])
df3.to_csv('data/merged_data.csv', index=False)

### tool method, compare two csv file and print out differences

In [ ]:
with open('data/merged_data.csv', 'r') as t1, open('data/joined.csv', 'r') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()

with open('data/update.csv', 'w') as outFile:
    for line in filetwo:
        if line not in fileone:
            outFile.write(line)

## 3. distribute label columns

In [5]:
df = pd.read_csv('data/merged_data.csv')
print(df)

              日期          送端          受端  平均气温/0.1℃  平均相对湿度/ 1%   \
0     2011-11-01  133427.912  127984.718        135           71   
1     2011-11-02  136135.276  129876.702        131           73   
2     2011-11-03  146135.962  140753.688        123           77   
3     2011-11-04  148216.315  142729.717        128           78   
4     2011-11-05  130378.126  125178.466        140           78   
5     2011-11-06  117725.187  113162.113        129           81   
6     2011-11-07  134108.701  129606.133        118           95   
7     2011-11-08  124136.145  119198.374        106           95   
8     2011-11-09  101445.615   97613.878        102           77   
9     2011-11-10  100960.687   97407.986        101           79   
10    2011-11-11  102897.187   99250.596        115           79   
11    2011-11-12   95233.820   91657.624        121           77   
12    2011-11-13   92857.987   90381.802        127           73   
13    2011-11-14  113357.215  109336.638        

In [7]:
for i in range(7, len(df.columns)):
    df_temp = df.drop(df.columns[list(range(7, len(df.columns)))], axis=1)
    df_temp[df.columns[i]] = df[df.columns[i]]
    df_temp.to_csv('data/selected/def' + str(i - 6) + '.csv', index=False)

### 4. make the label 0 or 1

In [9]:
from os import listdir

In [23]:
files = listdir('data/selected')
for file in files:
    df = pd.read_csv('data/selected/' + file)
    df[df.columns[-1]] = df[df.columns[-1]].apply(lambda x: 1 if x > 1 else 0)
    df.to_csv('data/labeled/' + file, index=False)